In [13]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from os.path import join as oj
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
results_dir = '/accounts/projects/vision/chandan/class-weight-uncertainty/results/pmlb'
os.makedirs(results_dir, exist_ok=True)

rs = []
for name in ['logistic', 'mlp2', 'gb']:
    out_dir = '/scratch/users/vision/chandan/pacmed/no_flips/' + name
    fnames = sorted([fname for fname in os.listdir(out_dir)])
    results_list = [pd.Series(pkl.load(open(oj(out_dir, fname), 
                                            "rb"))) for fname in tqdm(fnames)]
    rs.append(pd.concat(results_list, axis=1).T.infer_objects())

100%|██████████| 94/94 [00:00<00:00, 156.71it/s]


**note - cw is class-weight an pe is predictive-entropy**

# look at best dsets

In [23]:
for i in range(len(rs)):
    r = deepcopy(rs[i])
    r['auc_auc_diff'] = r.cw_auc_auc_test - r.pe_auc_auc_test
    r['calibration_diff'] = r.cw_calibration_rmse_test - r.pe_calibration_rmse_test
    rs[i] = r.sort_values(by='auc_auc_diff', ascending=False)

In [ ]:
for r in rs:
    print(r[['dset_name', 'auc_auc_diff']].head(n=15))

**pick best common dsets**

In [76]:
s0 = set(rs[0]['dset_name'][:15])
s1 = set(rs[1]['dset_name'][:15])
s2 = set(rs[2]['dset_name'][:15])

common_names = list(s0 & s1 & s2)
common_names += ['clean2', 'coil2000', 'Hill_Valley_with_noise', 
                 'hypothyroid', 'diabetes', 'agaricus-lepiota']
print('common_names', len(common_names), common_names)

common_names 10 ['tokyo1', 'churn', 'german', 'dis', 'clean2', 'coil2000', 'Hill_Valley_with_noise', 'hypothyroid', 'diabetes', 'agaricus-lepiota']


# print statements

In [89]:
for i in range(3):
    rs[i] = rs[i][rs[i].dset_name.isin(common_names)]

In [103]:
results = rs[0].merge(rs[1], how='outer', on='dset_name', 
                      suffixes=('', '_y'))
results = pd.merge(results, rs[2], how='outer', on='dset_name',
                   suffixes=('_x', '_z'))

In [104]:
# results.keys()

Index(['dset_name', 'seed_x', 'class_weight_x', 'model_type_x', 'flip_frac_x',
       'out_dir_x', 'pid_x', 'cw_flipped_diff_train_x',
       'cw_flipped_diff_p_train_x', 'cw_loss_percentages_train_x',
       ...
       'pe_loss_auc_test_z', 'pe_auc_percentages_test_z',
       'pe_auc_performances_test_z', 'pe_auc_auc_test_z',
       'pe_calibration_pred_test_z', 'pe_calibration_true_test_z',
       'pe_calibration_rmse_test_z', 'pe_ranks_test_z', 'auc_auc_diff_z',
       'calibration_diff_z'],
      dtype='object', length=169)

In [115]:
results.pe_auc_auc_test_x

0    76.181632
1    53.375482
2    42.871320
3    71.465215
4    48.399229
5    65.662951
6    58.796164
7    94.412386
8    57.660494
9    29.730631
Name: pe_auc_auc_test_x, dtype: float64

In [145]:
print('\\begin{tabular}{l c c c c c c}')
print('& \\multicolumn{2}{c}{Logistic Regression} &\t \\multicolumn{2}{c}{2-layer multilayer perceptron} &\t \\multicolumn{2}{c}{Gradient Boosting}\\\\')
print('Dataset	&	Class-weight &	Baseline &	Class-weight &	Baseline &	Class-weight &	Baseline\\\\')
print('\hline')
vals = [] 
thresh = 8
bf = '\\textbf{'
for name in sorted(common_names):
    row = results[results.dset_name == name].iloc[0]
    name = row.dset_name.replace("_", "\_")
    s = f'{name}\t&'
    if row.cw_auc_auc_test_x > row.pe_auc_auc_test_x + thresh:
        s += f'\t{bf}{row.cw_auc_auc_test_x:0.2f} }} \t&\t{row.pe_auc_auc_test_x:0.2f} \t&'
    else:
        s += f'\t{row.cw_auc_auc_test_x:0.2f}\t&\t{row.pe_auc_auc_test_x:0.2f} \t&'
        
    if row.cw_auc_auc_test_y > row.pe_auc_auc_test_y + thresh:
        s += f'\t{bf}{row.cw_auc_auc_test_y:0.2f} }} \t&\t{row.pe_auc_auc_test_y:0.2f} \t&'
    else:
        s += f'\t{row.cw_auc_auc_test_y:0.2f}\t&\t{row.pe_auc_auc_test_y:0.2f} \t&'
        
    if row.cw_auc_auc_test_z > row.pe_auc_auc_test_z + thresh:
        s += f'\t{bf}{row.cw_auc_auc_test_z:0.2f} }} \t&\t{row.pe_auc_auc_test_z:0.2f}\\\\'
    else:
        s += f'\t{row.cw_auc_auc_test_z:0.2f}\t&\t{row.pe_auc_auc_test_z:0.2f}\\\\'
    print(s)    
print('\end{tabular}')

\begin{tabular}{l c c c c c c}
& \multicolumn{2}{c}{Logistic Regression} &	 \multicolumn{2}{c}{2-layer multilayer perceptron} &	 \multicolumn{2}{c}{Gradient Boosting}\\
Dataset	&	Class-weight &	Baseline &	Class-weight &	Baseline &	Class-weight &	Baseline\\
\hline
Hill\_Valley\_with\_noise	&	74.82	&	71.47 	&	\textbf{64.38 } 	&	54.16 	&	45.42	&	45.23\\
agaricus-lepiota	&	94.42	&	94.41 	&	96.76	&	96.76 	&	\textbf{96.76 } 	&	67.70\\
churn	&	\textbf{61.62 } 	&	53.38 	&	\textbf{76.01 } 	&	65.20 	&	86.62	&	81.82\\
clean2	&	\textbf{96.00 } 	&	76.18 	&	\textbf{93.58 } 	&	36.00 	&	15.88	&	17.15\\
coil2000	&	57.36	&	57.66 	&	55.11	&	52.58 	&	\textbf{74.46 } 	&	64.32\\
diabetes	&	59.75	&	58.80 	&	51.60	&	49.57 	&	56.71	&	57.62\\
dis	&	48.96	&	42.87 	&	\textbf{70.42 } 	&	37.65 	&	\textbf{50.09 } 	&	41.88\\
german	&	51.11	&	48.40 	&	\textbf{50.61 } 	&	28.45 	&	59.98	&	52.50\\
hypothyroid	&	26.30	&	29.73 	&	52.97	&	57.60 	&	\textbf{61.79 } 	&	45.26\\
tokyo1	&	67.55	&	65.66 	&	68.39	&	65.65 	&	\textbf